In [11]:
import numpy as np
import torch
from PIL import Image
import pickle
from torchvision import transforms  # Import transforms

In [38]:
pip install torch numpy pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Haris\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [39]:
import torchvision.transforms as transforms

def load_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Assume karen ke SimCLR model ko 224x224 size ki images ki zarurat hai
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image


In [40]:
# Load the Random Forest model
with open('random_forest_model.pkl', 'rb') as f:
    loaded_rf_model = pickle.load(f)


In [41]:
import torch

def predict_class(image_tensor, model_path='random_forest_model.pkl'):
    # Load the trained model
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    model.eval()
    with torch.no_grad():
        features = model.encoder(image_tensor)  # Assuming encoder is the right method
        predictions = model.classifier(features)  # Yeh step model par depend karta hai ke classifier kaise defined hai
    return predictions.argmax(dim=1)  # Assuming ke output logits hain


In [57]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

# Assume you have a directory 'training_data' with subdirectories for each class
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'Train_resized_images/',
    target_size=(224, 224),
    batch_size=32,  # Adjust based on your system's memory
    class_mode='categorical',
    shuffle=True
)

# Load EfficientNetB0 without the top layer to extract features
base_model = EfficientNetB0(include_top=False, weights='imagenet', pooling='avg')

# Extract features
features = []
for i in range(len(train_generator)):
    images, _ = train_generator.next()
    features_batch = base_model.predict(images)
    features.append(features_batch)

# Combine all extracted features
training_features = np.vstack(features)


Found 99199 images belonging to 6 classes.


AttributeError: 'DirectoryIterator' object has no attribute 'next'

In [56]:
# Assuming you have access to training data with multiple samples
# Fit PCA on training data
pca = PCA(n_components=64)
pca.fit(training_features)  # Where training_features would be the high-dimensional features of your training set
pickle.dump(pca, open('pca_model.pkl', 'wb'))  # Save the fitted PCA model


NameError: name 'training_features' is not defined

In [55]:
import numpy as np
import pickle
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.decomposition import PCA

# Function to extract features and reduce dimensions
def extract_features(img_path, num_features=64):
    model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    
    # Reduce dimensions
    pca = PCA(n_components=num_features)
    reduced_features = pca.fit_transform(features.flatten().reshape(1, -1))
    
    return reduced_features.flatten()

# Modify the predict_image_class function to use the updated feature extraction
def predict_image_class(img_path, rf_model, class_indices, threshold=0.6):
    # Extract features
    features = extract_features(img_path, num_features=64)  # Ensure this matches the RF model expectation

    # Make prediction using the Random Forest model
    predictions = rf_model.predict_proba([features])[0]
    predicted_class_index = np.argmax(predictions)
    confidence = np.max(predictions)

    # Reverse the class indices dictionary to get class names
    labels = {v: k for k, v in class_indices.items()}

    # Check confidence threshold
    if confidence < threshold:
        return "Unknown"
    else:
        return labels.get(predicted_class_index, "Unknown")
# Load the Random Forest model from a .pkl file
with open('random_forest_model.pkl', 'rb') as f:
    loaded_rf_model = pickle.load(f)

# Example usage:
class_indices = {0: "F", 1: "M", 2: "N", 3: "Q", 4: "S", 5: "V"}  # Adjust as per your actual classes
img_path = 'ECG_Image_data/test/F/F6.png'
predicted_class = predict_image_class(img_path, loaded_rf_model, class_indices, threshold=0.7)
print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step


ValueError: n_components=64 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [6]:
# Re-define the SimCLR-related classes
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

class ProjectionHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ProjectionHead, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)
        self.fc2 = nn.Linear(output_dim, output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class SimCLR(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, projection_dim):
        super(SimCLR, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, output_dim)
        self.projection_head = ProjectionHead(output_dim, projection_dim)
    
    def forward(self, x):
        h = self.encoder(x)
        z = self.projection_head(h)
        return h, z


In [7]:
with open('simclr_model.pkl', 'rb') as f:
    loaded_simclr_model = pickle.load(f)


In [8]:
import numpy as np
import torch

# Load the test data
X_test = np.load('X_test.npy')

# Load the saved Random Forest model
with open('random_forest_model.pkl', 'rb') as f:
    loaded_rf_model = pickle.load(f)

# Load the trained SimCLR model (for feature extraction)
with open('simclr_model.pkl', 'rb') as f:
    loaded_simclr_model = pickle.load(f)

# Convert test data to PyTorch tensor for feature extraction
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Extract features from the test data using the loaded SimCLR model
def extract_features(model, X):
    model.eval()
    with torch.no_grad():
        h = model.encoder(X)  # Extract features using the encoder
    return h

X_test_features = extract_features(loaded_simclr_model, X_test_tensor)

# Perform inference with the loaded Random Forest model
y_pred_test = loaded_rf_model.predict(X_test_features.numpy())

# Print predictions
print("Predictions:", y_pred_test)

# If you have ground-truth labels and want to compute accuracy
y_test = np.load('y_test.npy')  # Load test labels
from sklearn.metrics import accuracy_score

test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Predictions: [3 4 5 ... 3 2 2]
Test Accuracy: 99.94%


In [20]:
class SimCLR(nn.Module):
    def __init__(self, encoder):
        super(SimCLR, self).__init__()
        self.encoder = encoder  # Use the modified encoder

    def forward(self, x):
        return self.encoder(x)


In [21]:
# Instantiate the SimCLR model with the modified encoder
modified_simclr_model = SimCLR(encoder=ModifiedEncoder())


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image

# Load the pre-trained ResNet model (e.g., ResNet-18)
resnet_model = models.resnet18(pretrained=True)

# Modify ResNet for feature extraction (remove the final fully connected layer)
resnet_model = nn.Sequential(*list(resnet_model.children())[:-1])  # Remove the last fully connected layer

# Set the model to evaluation mode
resnet_model.eval()

# Define the Modified Encoder with the correct input size (512)
class ModifiedEncoder(nn.Module):
    def __init__(self):
        super(ModifiedEncoder, self).__init__()
        self.fc1 = nn.Linear(512, 128)  # Change input size from 100 to 512
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)  # Output size of 64

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Define SimCLR model class
class SimCLR(nn.Module):
    def __init__(self, encoder):
        super(SimCLR, self).__init__()
        self.encoder = encoder  # Use the modified encoder

    def forward(self, x):
        return self.encoder(x)

# Instantiate the SimCLR model with the modified encoder
modified_simclr_model = SimCLR(encoder=ModifiedEncoder())

# Example feature extraction and prediction
def preprocess_image(image_path):
    # Define the image transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Open and transform the image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

def extract_cnn_features(image):
    # Use ResNet model to extract features
    with torch.no_grad():
        features = resnet_model(image)  # ResNet features (512-dimensional vector)
        features = features.view(features.size(0), -1)  # Flatten to (batch_size, 512)
    return features

def extract_encoder_features(model, cnn_features):
    model.eval()
    with torch.no_grad():
        features = model.encoder(cnn_features)  # Pass features through the modified encoder
    return features

def predict_image_class(image_path):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)
    
    # Extract features using the CNN backbone (ResNet)
    cnn_features = extract_cnn_features(preprocessed_image)
    
    # Extract features from the encoder using the CNN output
    encoder_features = extract_encoder_features(modified_simclr_model, cnn_features)
    
    # Predict the class using the Random Forest model
    predicted_class = loaded_rf_model.predict(encoder_features.numpy())
    
    return predicted_class[0]

# Example usage
image_path = "ECG_Image_data/test/F/F67.png"  # Replace with the path to your image
predicted_class = predict_image_class(image_path)
print(f"Predicted Class: {predicted_class}")


C:\Users\Haris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Haris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicted Class: 2


In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Assuming you are using SimCLR with a CNN backbone (e.g., ResNet)
class SimCLR(nn.Module):
    def __init__(self, cnn_backbone, projection_dim=128):
        super(SimCLR, self).__init__()
        self.encoder = cnn_backbone  # CNN backbone like ResNet
        self.projection_head = nn.Sequential(
            nn.Linear(2048, 512),  # Adjust this based on the output size of your CNN
            nn.ReLU(),
            nn.Linear(512, projection_dim)
        )
        
    def forward(self, x):
        # Pass through the encoder (CNN part, e.g., ResNet)
        h = self.encoder(x)  # output from the CNN backbone
        
        # Flatten the CNN output before passing it to the projection head
        h = torch.flatten(h, 1)  # Flatten the tensor into a 1D vector (batch_size, flattened_size)
        
        # Pass through the projection head (fully connected layers)
        z = self.projection_head(h)
        return h, z

# Example feature extraction function
def extract_features(model, X):
    model.eval()
    with torch.no_grad():
        # Pass through the entire model
        cnn_features, projection_features = model(X)  # Features after the encoder and projection head
    return projection_features

# Example usage
preprocessed_image = preprocess_image(image_path)  # Ensure image is preprocessed properly
image_features = extract_features(loaded_simclr_model, preprocessed_image)

# Now you can use these features for prediction
predicted_probs = loaded_rf_model.predict_proba(image_features.numpy())  # Use the extracted features
predicted_class = predicted_probs.argmax(axis=1)[0]
print(f"Predicted Class: {predicted_class}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (672x224 and 100x128)

In [85]:
import numpy as np
import scipy.stats

def custom_feature_extraction(image):
    # Assume image is preprocessed and normalized to 224x224
    # Let's create a simple feature set:
    # - Average, variance, skewness, and kurtosis for each color channel

    features = []
    for channel in range(image.shape[2]):  # Assuming last dimension is channel
        channel_data = image[:, :, channel].flatten()
        features.append(np.mean(channel_data))
        features.append(np.var(channel_data))
        features.append(scipy.stats.skew(channel_data))
        features.append(scipy.stats.kurtosis(channel_data))
    
    # You may need to adjust this to exactly get 64 features, this is just a starting point
    # Ensure the final features array has 64 elements
    features = np.array(features)
    if len(features) > 64:
        features = features[:64]  # Truncate to the first 64 features
    elif len(features) < 64:
        # Pad with zeros if not enough features (not ideal, but for demonstration)
        features = np.pad(features, (0, 64 - len(features)), mode='constant')
    
    return features.reshape(1, -1)  # Reshape to 2D array for scikit-learn

# Extract features
features = custom_feature_extraction(image_array)

# Load the model and predict
import pickle
model_path = 'random_forest_model.pkl'
with open(model_path, 'rb') as file:
    model = pickle.load(file)

predictions = model.predict(features)
print("Predictions:", predictions)


C:\Users\Haris\AppData\Local\Temp\ipykernel_3528\2075653337.py:14: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(scipy.stats.skew(channel_data))
C:\Users\Haris\AppData\Local\Temp\ipykernel_3528\2075653337.py:15: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(scipy.stats.kurtosis(channel_data))


Predictions: [2]


In [99]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
import numpy as np
import pickle

# Load the pretrained VGG16 model from Keras applications
base_model = VGG16(weights='imagenet', include_top=False)
model_cnn = Model(inputs=base_model.input, outputs=base_model.output)

# Prepare the image
image_path = 'Train_resized_images/resized_images_of_F/resized_image_2.png'  # Update with the actual path to your image
image = load_img(image_path, target_size=(224, 224))  # Resize image
image_array = img_to_array(image)  # Convert image to array
image_array = np.expand_dims(image_array, axis=0)  # Add a batch dimension
image_array = preprocess_input(image_array)  # Preprocess the image

# Use the CNN model to extract features
features = model_cnn.predict(image_array)

# Use GlobalAveragePooling2D to reduce dimensions
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
features = global_average_layer(features)  # Apply pooling to reduce dimensions

# Convert to a 1D tensor to mimic flattening
features = tf.reshape(features, shape=[-1])

# Ensure you have 64 features, adjust if necessary
if tf.shape(features)[0] < 64:
    # Pad the features if there are less than 64
    features = tf.pad(features, [[0, 64 - tf.shape(features)[0]]])
else:
    # Truncate the features if there are more than 64
    features = features[:64]

features = tf.reshape(features, shape=[1, -1])  # Reshape to make it 2D (1 sample, 64 features)

# Load your RandomForest model
with open('random_forest_model.pkl', 'rb') as file:
    model_rf = pickle.load(file)

# Predict with RandomForest
predictions = model_rf.predict(features.numpy())  # Convert tensor to numpy array for scikit-learn
print("Predictions:", predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predictions: [2]


In [78]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import cv2

def extract_color_histogram(image, bins=(8, 8, 8)):
    """ Extract a 3D color histogram from the HSV color space using the specified
        number of bins per channel """
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

# Load and preprocess the image
image_path = 'ECG_Image_data/test/V/V28.png'
image_size = (224, 224)  # Adjust to your model's input size
image = load_img(image_path, target_size=image_size)
image_array = img_to_array(image)
image_array = image_array / 255.0  # Normalize if your model was trained with normalization

# Extract features
image_features = extract_color_histogram(image_array.astype('uint8'))  # Make sure data type matches that expected by CV2

# Resize features to match expected input (if necessary, using interpolation techniques like the following simple one)
if len(image_features) > 64:
    # Interpolating down to the number of features expected by the model
    from scipy.interpolate import interp1d
    x = np.linspace(0, len(image_features), num=64, endpoint=False)
    f = interp1d(np.arange(len(image_features)), image_features, kind='linear')
    image_features = f(x)

# Predict using the model (assuming model is already loaded)
predictions = model.predict(image_features.reshape(1, -1))  # Reshape to 2D (1 sample, 64 features)
predicted_class_index = np.argmax(predictions)

# Define class labels based on your model's training
class_labels = ['F', 'M', 'N', 'Q', 'S', 'V']
predicted_class_label = class_labels[predicted_class_index]

print(f"Predicted Class: {predicted_class_label}")


Predicted Class: F
